In [42]:
#####継続時間と離散時間の同時分析のための変量効果モデル#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns
import time

#np.random.seed(20)

In [43]:
####データの発生####
##データの設定
hh = 10000   #ユーザー数
pt = np.random.poisson(np.random.gamma(15, 1/0.25, hh), hh)   #ユーザーあたりのレコード数
hhpt = np.sum(pt)   #総レコード数
dt = 200   #最大観測期間

In [44]:
##IDとインデックスの設定
#IDの設定
user_id = np.repeat(range(hh), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(hh)])))

#インデックスの設定
index = np.array(range(hhpt))
user_index = [i for i in range(hh)]
for i in range(hh):
    user_index[i] = index[user_id==i]

In [45]:
####説明変数の生成####
##素性ベクトルの生成
k1 = 6; k2 = 8; k3 = 6
x1 = np.random.uniform(0, 1, hhpt*k1).reshape(hhpt, k1)
x2 = np.zeros((hhpt, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.5, 1)
    x2[:, j] = np.random.binomial(1, prob, hhpt)
x3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(2.0, k3)), hhpt)
x3 = np.delete(x3, np.sum(x3, axis=0).argmin(), axis=1)
x = np.hstack((np.repeat(1, hhpt).reshape(hhpt, 1), x1, x2, x3))

##個体間説明変数の生成
k1 = 4; k2 = 5; k3 = 5
u1 = np.random.normal(0, 1, hh*k1).reshape(hh, k1)
u2 = np.zeros((hh, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.5, 1)
    u2[:, j] = np.random.binomial(1, prob, hh)
u3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(2.0, k3)), hh)
u3 = np.delete(u3, np.sum(u3, axis=0).argmin(), axis=1)
u = np.hstack((np.repeat(1, hh).reshape(hh, 1), u1, u2, u3))

In [46]:
####継続時間の応答変数を生成####
rp = 0
while True:
    rp = rp + 1

    ##パラメータの設定
    #モデルパラメータを生成
    alpha1 = np.log(np.random.uniform(0.7, 1.3, 1))
    beta1 = np.append(np.random.uniform(0, 1.2, 1), np.random.uniform(-0.6, 1.2, x.shape[1]-1))
    alphat1 = alpha1; betat1= beta1

    #変量効果を生成
    tau = np.random.uniform(0.5, 0.75, 1)
    theta = np.random.normal(0, tau, hh)   
    taut = tau; thetat = theta

    ##ワイブル分布からイベント時間を生成
    #ワイブル分布のを尺度パラメータを設定
    Lambda = np.exp(np.dot(x, beta1) + theta[user_id])

    #イベント時間を生成
    y_full = scipy.stats.weibull_min.rvs(np.exp(alpha1), 0, Lambda, hhpt)

    #停止条件
    if (np.max(y_full) < 1000) & (np.mean(y_full) > 5.0):
        break

In [47]:
##打ち切りの設定
y_list = [i for i in range(hh)]
z_list = [i for i in range(hh)]
x_list = [i for i in range(hh)]
user_id_list = [i for i in range(hh)]
pt_id_list = [i for i in range(hh)]
new_pt = np.repeat(0, hh)

for i in range(hh):
    #右側打ち切りのインデックスを作成
    y_dt = y_full[user_index[i]]
    index_censor = np.arange(pt[i])[np.cumsum(y_dt) < dt]

    #打ち切りパターンにより変数を設定
    if (pt[i] > len(index_censor)) & (len(index_censor) > 0):
        y_list[i] = np.append(y_dt[index_censor], dt - np.max(np.cumsum(y_dt[index_censor])))
        z_list[i] = np.append(np.repeat(0, np.max(index_censor) + 1), 1)
    elif len(index_censor)==pt[i]:
        y_list[i] = y_dt[index_censor]
        z_list[i] = np.repeat(0, pt[i])
    elif len(index_censor)==0:
        y_list[i] = np.array([dt])
        z_list[i] = np.array([1])

    #変数を格納
    n = len(y_list[i])
    index = user_index[i][np.arange(n)]
    new_pt[i] = n
    x_list[i] = x[index, ]
    user_id_list[i] = np.repeat(i, n) 
    pt_id_list[i] = np.arange(n)

In [48]:
#リストを変換
pt = new_pt
hhpt = np.sum(pt)
user_id = np.array(list(itertools.chain(*[user_id_list[i] for i in range(hh)])))
pt_id = np.array(list(itertools.chain(*[pt_id_list[i] for i in range(hh)])))
z = 1 - np.array(list(itertools.chain(*[z_list[i] for i in range(hh)])))
y1 = np.array(list(itertools.chain(*[y_list[i] for i in range(hh)])))
y_censorl = np.log(y1)   #継続時間を対数変換
x1 = np.array(list(itertools.chain(*[x_list[i] for i in range(hh)])))
k1 = x1.shape[1]

In [49]:
#インデックスを設定
index = np.arange(hhpt)
user_index = [i for i in range(hh)]
for i in range(hh):
    user_index[i] = index[user_id==i]
z_index = index[z==0]
user_dt = sparse.coo_matrix((np.repeat(1, hhpt), (user_id, range(hhpt))), shape=(hh, hhpt)).tocsr()

In [50]:
####離散選択の応答変数を生成####
rp = 0
while True:
    rp = rp + 1

    ##データの結合
    u_vec = theta[user_id]
    x2 = np.hstack((x1, u_vec.reshape(hhpt, 1)))
    k2 = x2.shape[1]

    ##パラメータの生成
    beta2 = np.append(-0.7, np.random.normal(0, 0.75, k2-1))
    betat2 = beta2
    
    ##ベルヌーイ分布から応答変数を生成
    #ロジットと応答確率
    logit = np.dot(x2, beta2)
    prob = np.exp(logit) / (1 + np.exp(logit))

    #応答変数を生成
    y2 = np.random.binomial(1, prob, hhpt)

    #停止条件
    if (np.mean(y2) < 0.4) & (np.mean(y2) > 0.2):
        break

In [51]:
####マルコフ連鎖モンテカルロ法で継続時間と離散選択を同時推定####
##ワイブルモデルの対数事後分布
def LLho_w1(gamma, theta, beta_mu, inv_Cov, x, y, y_censorl, z, user_id):
    #パラメータの設定
    alpha = np.exp(gamma[0])
    beta = np.delete(gamma, 0)

    #ワイブルモデルの対数尤度
    Lambda = np.dot(x, beta) + theta[user_id]
    scale = (y_censorl - Lambda) / alpha
    LL_weibull = np.sum(z * (-np.log(alpha) + scale) - np.exp(scale))
    
    #多変量正規分布の対数事前分布
    er = gamma - beta_mu
    LL_mvn = -1/2 * np.dot(np.dot(er, inv_Cov), er)
    
    #対数事後分布の和
    LL = LL_weibull + LL_mvn
    return LL

In [52]:
##ワイブルモデルのユーザーごとの対数事後分布の和
def LLho_w2(theta, gamma, beta_mu, tau, x, y, y_censorl, z, user_dt, user_id, hhpt):
    #パラメータの設定
    alpha = np.exp(gamma[0])
    beta = np.delete(gamma, 0)

    #ワイブルモデルの対数尤度
    Lambda = np.dot(x, beta) + theta[user_id]
    scale = (y_censorl - Lambda) / alpha
    LLi_weibull = z * (-np.log(alpha) + scale) - np.exp(scale)
    
    #正規分布の対数事前分布
    er = theta - beta_mu
    LLi_normal = -1/2 * np.power(er, 2) / tau
    
    #ユーザーごとの対数事後分布の和
    LLi = np.array(np.dot(user_dt, sparse.csr_matrix(LLi_weibull.reshape(hhpt, 1))).todense()).reshape(-1) + LLi_normal
    return LLi

In [53]:
##ロジスティック回帰モデルの対数事後分布
def LLho_logit(beta, beta_mu, inv_Cov, x, y, hhpt):
    #ロジットと応答確率を設定
    mu = np.exp(np.dot(x, beta))
    Prob = mu / (1 + mu)

    #対数尤度と対数事前分布を設定
    er = beta - beta_mu
    LL_logit = y*np.log(Prob) + (1-y)*np.log(1-Prob) 
    LL_mvn = -1/2 * np.dot(np.dot(er, inv_Cov), er)

    #対数事後分布の和
    LL = np.sum(LL_logit) + LL_mvn
    return LL

In [54]:
##ワイブルモデルのモデルパラメータの勾配ベクトル
def LLd_w1(gamma, theta, beta_mu, inv_Cov, x, y, y_censorl, z, user_id):
    #パラメータの設定
    alpha = np.exp(gamma[0])
    beta = np.delete(gamma, 0)

    #尺度パラメータの勾配係数
    Lambda = np.dot(x, beta) + theta[user_id]
    scale = (y_censorl - Lambda) / alpha; scale_exp = np.exp(scale)
    LLd1 = -np.sum(z)/alpha + np.sum(-scale/alpha * (z - scale_exp))

    #回帰パラメータの勾配ベクトル
    LLd2 = np.sum(-x/alpha * (z - scale_exp).reshape(hhpt, 1), axis=0)

    #対数事前分布の勾配ベクトル
    er = gamma - beta_mu
    dmvn = -np.dot(inv_Cov1, er.T)   

    #勾配ベクトルを結合
    LLd = -(np.append(LLd1, LLd2) + dmvn)
    return LLd

In [55]:
##ワイブルモデルのランダム効果の勾配係数
def LLd_w2(theta, gamma, beta_mu, tau, x, y, y_censorl, z, user_dt, user_id):
    #パラメータの設定
    alpha = np.exp(gamma[0])
    beta = np.delete(gamma, 0)

    #ランダム効果の勾配係数
    Lambda = np.dot(x, beta) + theta[user_id]
    scale = (y_censorl - Lambda) / alpha; scale_exp = np.exp(scale)
    LLd = -1/alpha * (z - scale_exp).reshape(hhpt, 1)
    
    #対数事前分布の勾配係数
    er = theta - beta_mu
    dnormal = -er / tau
    
    #ユーザーごとの対数事後分布の勾配係数の和
    LLd = -(np.array(np.dot(user_dt, sparse.csr_matrix(LLd)).todense()).reshape(-1) + dnormal)
    return LLd

In [56]:
##ロジスティック回帰モデルの勾配ベクトル
def LLd_logit(beta, beta_mu, inv_Cov, x, y, hhpt):
    #ロジットと応答確率を設定
    mu = np.exp(np.dot(x, beta))
    Prob = mu / (1 + mu)

    #勾配ベクトルの設定
    er = beta - beta_mu
    dlogit = np.sum(y.reshape(hhpt, 1)*x - Prob.reshape(hhpt, 1)*x, axis=0)
    dmvn = -np.dot(inv_Cov, er)
    LLd = -(dlogit + dmvn)
    return LLd

In [57]:
##ワイブルモデルのモデルパラメータのリープフロッグ法を解く関数
def leapfrog_w1(r, z1, D, e, L): 
    def leapfrog_step(r, z1, e):
        r2 = r - e * D(z1, theta, beta_mu1, inv_Cov1, x1, y1, y_censorl, z, user_id) / 2
        z2 = z1 + e * r2
        r2 = r2 - e * D(z2, theta, beta_mu1, inv_Cov1, x1, y1, y_censorl, z, user_id) / 2
        return [r2, z2]   #1回の移動後の運動量と座標
    leapfrog_result = [r, z1]
    for i in range(L):
        leapfrog_result = leapfrog_step(leapfrog_result[0], leapfrog_result[1], e)
    return leapfrog_result

In [58]:
##ワイブルモデルのランダム効果のリープフロッグ法を解く関数
def leapfrog_w2(r, z1, D, e, L): 
    def leapfrog_step(r, z1, e):
        r2 = r - e * D(z1, gamma, theta_mu, tau, x1, y1, y_censorl, z, user_dt, user_id) / 2
        z2 = z1 + e * r2
        r2 = r2 - e * D(z2, gamma, theta_mu, tau, x1, y1, y_censorl, z, user_dt, user_id) / 2
        return [r2, z2]   #1回の移動後の運動量と座標
    leapfrog_result = [r, z1]
    for i in range(L):
        leapfrog_result = leapfrog_step(leapfrog_result[0], leapfrog_result[1], e)
    return leapfrog_result

In [59]:
##ロジスティック回帰モデルの回帰ベクトルのリープフロッグ法を解く関数
def leapfrog_logit(r, z1, D, e, L): 
    def leapfrog_step(r, z1, e):
        r2 = r - e * D(z1, beta_mu2, inv_Cov2, x2, y2, hhpt) / 2
        z2 = z1 + e * r2
        r2 = r2 - e * D(z2, beta_mu2, inv_Cov2, x2, y2, hhpt) / 2
        return [r2, z2]   #1回の移動後の運動量と座標
    leapfrog_result = [r, z1]
    for i in range(L):
        leapfrog_result = leapfrog_step(leapfrog_result[0], leapfrog_result[1], e)
    return leapfrog_result

In [61]:
##アルゴリズムの設定
LL1 = -100000000   #対数尤度の初期値
R = 2000
keep = 2  
iter = 0
burnin = int(250/keep)
disp = 10
e_w1 = np.append(0.000025, np.repeat(0.00025, k1))
e_w2 = 0.05
e = 0.001
L = 3

In [62]:
##事前分布の設定
#階層モデルの事前分布
theta_mu = 0
s0 = 0.25; v0 = 0.25

#モデルパラメータの事前分布
beta_mu1 = np.repeat(0, k1+1) 
inv_Cov1 = np.linalg.inv(np.diag(np.repeat(100, k1+1)))
beta_mu2 = np.repeat(0, k2) 
inv_Cov2 = np.linalg.inv(np.diag(np.repeat(100, k2)))

In [63]:
##パラメータの真値
#階層モデルの真値
tau = taut
inv_tau = 1/tau

#ワイブルモデルの真値
alpha = alphat1
beta1 = betat1
gamma = np.append(alpha, beta1)
theta = thetat

#ロジスティックモデルの真値
beta2 = betat2

In [64]:
##パラメータの初期値
#階層モデルの初期値
tau = 0.5
inv_tau = 1/tau

#ワイブルモデルの初期値
alpha = 0
beta1 = np.dot(np.dot(np.linalg.inv(np.dot(x1.T, x1)), x1.T), np.log(y1))
gamma = np.append(alpha, beta1)
theta = np.random.normal(0, 0.1, hh)

#ロジスティックモデルの初期値
u_vec = theta[user_id]
x2 = np.hstack((x1, u_vec.reshape(hhpt, 1)))
beta2 = np.dot(np.dot(np.linalg.inv(np.dot(x2.T, x2)), x2.T), y2)

In [65]:
##パラメータの格納法配列
ALPHA = np.zeros(int(R/keep))
BETA1 = np.zeros((int(R/keep), k1))
BETA2 = np.zeros((int(R/keep), k2))
THETA = np.zeros((int(R/keep), hh))

In [66]:
##真値での対数尤度
LLbest1 = LLho_w1(np.append(alphat1, betat1), thetat, beta_mu1, inv_Cov1, x1, y1, y_censorl, z, user_id)
LLbest2 = LLho_logit(betat2, beta_mu2, inv_Cov2, np.hstack((x1, thetat[user_id].reshape(hhpt, 1))), y2, hhpt)
LLbest = LLbest1 + LLbest2
print(np.array([LLbest, LLbest1, LLbest2]))

[-777660.46418112 -615380.15139857 -162280.31278254]


In [67]:
####ハミルトニアンモンテカルロ法でパラメータをサンプリング####
for rp in range(R):
    
    ##ワイブルモデルのパラメータをサンプリング
    #HMCの新しいパラメータを生成
    rold = np.random.multivariate_normal(np.repeat(0, k1+1), np.diag(np.ones(k1+1)), 1).reshape(-1)
    gammad = gamma

    #リープフロッグ法による1ステップ移動
    res = leapfrog_w1(rold, gammad, LLd_w1, e_w1, L)
    rnew = res[0]
    gamman = res[1]

    #移動前と移動後のハミルトニアン
    Hnew = -LLho_w1(gamman, theta, beta_mu1, inv_Cov1, x1, y1, y_censorl, z, user_id) + np.sum(np.power(rnew, 2)) / 2
    Hold = -LLho_w1(gammad, theta, beta_mu1, inv_Cov1, x1, y1, y_censorl, z, user_id) + np.sum(np.power(rold, 2)) / 2

    #新しいパラメータの採択を決定
    rand = np.random.uniform(0, 1, 1)
    accept_prob = np.min(np.append(1, np.exp(Hold - Hnew)))   #採択率
    accept_prob1 = accept_prob

    #採択率に基づきgammaを採択
    if(accept_prob >= rand):
        gamma = gamman
    else:
        gamma = gammad


    ##ワイブルモデルのランダム効果をサンプリング
    rold = np.random.normal(0, 1, hh)
    thetad = theta

    #リープフロッグ法による1ステップ移動
    res = leapfrog_w2(rold, thetad, LLd_w2, e_w2, L)
    rnew = res[0]
    thetan = res[1]

    #移動前と移動後のハミルトニアン
    Hnew = -LLho_w2(thetan, gamma, theta_mu, tau, x1, y1, y_censorl, z, user_dt, user_id, hhpt) + np.power(rnew, 2) / 2
    Hold = -LLho_w2(thetad, gamma, theta_mu, tau, x1, y1, y_censorl, z, user_dt, user_id, hhpt) + np.power(rold, 2) / 2

    #新しいパラメータの採択を決定
    rand = np.random.uniform(0, 1, hh)
    accept_prob = np.min(np.hstack((np.repeat(1, hh).reshape(hh, 1), np.exp(Hold - Hnew).reshape(hh, 1))), axis=1)
    accept_prob2 = accept_prob

    #採択率に基づきthetaを採択
    flag = np.array(accept_prob > rand, dtype="int")
    theta = flag*thetan + (1-flag)*thetad

    
    ##ロジスティックモデルのパラメータをサンプリング
    #継続時間のランダム効果から新しい説明変数を作成
    u_vec = theta[user_id]
    x2 = np.hstack((x1, u_vec.reshape(hhpt, 1)))

    #HMCの新しいパラメータを生成
    rold = np.random.multivariate_normal(np.repeat(0, k2), np.diag(np.ones(k2)), 1).reshape(-1)
    betad2 = beta2

    #リープフロッグ法による1ステップ移動
    res = leapfrog_logit(rold, betad2, LLd_logit, e, L)
    rnew = res[0]
    betan2 = res[1]

    #移動前と移動後のハミルトニアン
    Hnew = -LLho_logit(betan2, beta_mu2, inv_Cov2, x2, y2, hhpt) + np.sum(np.power(rnew, 2)) / 2
    Hold = -LLho_logit(betad2, beta_mu2, inv_Cov2, x2, y2, hhpt) + np.sum(np.power(rold, 2)) / 2

    #新しいパラメータの採択を決定
    rand = np.random.uniform(0, 1, 1)
    accept_prob = np.min(np.append(1, np.exp(Hold - Hnew)))   #採択率
    accept_prob3 = accept_prob

    #採択率に基づきgammaを採択
    if(accept_prob >= rand):
        beta2 = betan2
    else:
        beta2 = betad2


    ##ワイブルモデルの階層モデルの分散をサンプリング
    #ガンマ分布のパラメータ
    s = np.sum(np.power(theta - np.mean(theta), 2)) + s0
    v = hh + v0

    #逆ガンマ分布からパラメータをサンプリング
    tau = np.sqrt(1/np.random.gamma(v/2, 1/(s/2), 1))
    
    
    ##サンプリング結果の格納と表示
    if rp%keep==0:
        mkeep = int(rp/keep)
        ALPHA[mkeep] = gamma[0]
        BETA1[mkeep, ] = np.delete(gamma, 0)
        BETA2[mkeep, ] = beta2
        THETA[mkeep, ] = theta

    if rp%disp==0:
        #対数尤度を更新
        LL1 = LLho_w1(gamma, theta, beta_mu1, inv_Cov1, x1, y1, y_censorl, z, user_id)
        LL2 = LLho_logit(beta2, beta_mu2, inv_Cov2, np.hstack((x1, thetat[user_id].reshape(hhpt, 1))), y2, hhpt)
        LL = LL1 + LL2
        
        #サンプリング結果を表示
        print(rp)
        print(np.round([accept_prob1, np.mean(accept_prob2), accept_prob3], 3))
        print(np.round(np.array([LL, LL1, LL2, LLbest, LLbest1, LLbest2]), 1))
        print(np.round(np.vstack((np.append(gamma, tau), np.append(np.append(alphat1, betat1), taut))), 3))
        print(np.round(np.vstack((beta2, betat2)), 3))

0
[1.    0.997 1.   ]
[-860942.  -647250.8 -213691.2 -777660.5 -615380.2 -162280.3]
[[ 0.002 -0.399  0.844  0.076  0.71  -0.341 -0.428 -0.523  0.787  0.994
   0.002  0.251  0.563 -0.174  0.154 -0.374  0.248  0.027  0.552  1.012
   0.246  0.303]
 [ 0.006  0.347  0.815  0.009  0.67  -0.404 -0.501 -0.587  0.749  0.987
  -0.044  0.199  0.543 -0.221  0.109 -0.431  0.21  -0.005  0.526  1.006
   0.191  0.669]]
[[-0.127 -0.254 -0.176 -0.06  -0.438 -0.209 -0.221 -0.144 -0.19  -0.183
  -0.063 -0.41  -0.28  -0.191 -0.063  0.011 -0.146  0.108  0.081 -0.155
   0.044]
 [-0.7   -0.302  0.222  0.986 -1.578 -0.004 -0.049  0.228 -0.505 -0.144
   0.769 -1.848 -0.936  0.013  0.644  0.38  -0.321  0.937  0.818  0.053
   1.753]]
10
[1.    0.993 1.   ]
[-796927.3 -617262.5 -179664.8 -777660.5 -615380.2 -162280.3]
[[ 0.002 -0.351  0.86   0.095  0.727 -0.323 -0.413 -0.51   0.797  1.003
   0.007  0.256  0.575 -0.171  0.161 -0.372  0.252  0.034  0.56   1.024
   0.254  0.656]
 [ 0.006  0.347  0.815  0.009  0.67  -

110
[1.    0.993 1.   ]
[-783214.3 -615998.8 -167215.5 -777660.5 -615380.2 -162280.3]
[[ 0.002 -0.248  0.867  0.084  0.732 -0.329 -0.426 -0.514  0.772  1.
  -0.019  0.232  0.56  -0.202  0.125 -0.407  0.263  0.044  0.569  1.055
   0.242  0.693]
 [ 0.006  0.347  0.815  0.009  0.67  -0.404 -0.501 -0.587  0.749  0.987
  -0.044  0.199  0.543 -0.221  0.109 -0.431  0.21  -0.005  0.526  1.006
   0.191  0.669]]
[[-0.533 -0.336  0.09   0.738 -1.441 -0.072 -0.159  0.215 -0.483 -0.153
   0.692 -1.759 -0.906  0.002  0.607  0.049 -0.606  0.569  0.477 -0.266
   1.528]
 [-0.7   -0.302  0.222  0.986 -1.578 -0.004 -0.049  0.228 -0.505 -0.144
   0.769 -1.848 -0.936  0.013  0.644  0.38  -0.321  0.937  0.818  0.053
   1.753]]
120
[1.    0.993 1.   ]
[-782873.3 -615946.9 -166926.4 -777660.5 -615380.2 -162280.3]
[[ 1.000e-03 -2.360e-01  8.670e-01  8.200e-02  7.330e-01 -3.310e-01
  -4.240e-01 -5.140e-01  7.700e-01  9.970e-01 -2.100e-02  2.300e-01
   5.590e-01 -2.010e-01  1.230e-01 -4.080e-01  2.660e-01  4.300

210
[0.981 0.993 0.994]
[-781637.3 -615725.4 -165911.9 -777660.5 -615380.2 -162280.3]
[[ 0.    -0.163  0.853  0.067  0.729 -0.342 -0.435 -0.528  0.763  0.996
  -0.027  0.223  0.553 -0.206  0.128 -0.418  0.281  0.045  0.575  1.06
   0.254  0.689]
 [ 0.006  0.347  0.815  0.009  0.67  -0.404 -0.501 -0.587  0.749  0.987
  -0.044  0.199  0.543 -0.221  0.109 -0.431  0.21  -0.005  0.526  1.006
   0.191  0.669]]
[[-0.746 -0.328  0.186  0.908 -1.538 -0.046 -0.073  0.236 -0.472 -0.143
   0.72  -1.803 -0.904  0.005  0.626  0.135 -0.545  0.68   0.553 -0.18
   1.554]
 [-0.7   -0.302  0.222  0.986 -1.578 -0.004 -0.049  0.228 -0.505 -0.144
   0.769 -1.848 -0.936  0.013  0.644  0.38  -0.321  0.937  0.818  0.053
   1.753]]
220
[0.996 0.993 0.996]
[-781521.5 -615773.1 -165748.5 -777660.5 -615380.2 -162280.3]
[[-1.000e-03 -1.560e-01  8.540e-01  6.900e-02  7.300e-01 -3.430e-01
  -4.280e-01 -5.300e-01  7.630e-01  9.930e-01 -3.000e-02  2.210e-01
   5.530e-01 -2.020e-01  1.260e-01 -4.180e-01  2.820e-01  4.40

310
[0.991 0.993 1.   ]
[-780676.5 -615507.6 -165168.9 -777660.5 -615380.2 -162280.3]
[[-1.000e-03 -8.900e-02  8.490e-01  5.300e-02  7.270e-01 -3.550e-01
  -4.440e-01 -5.440e-01  7.620e-01  9.880e-01 -3.200e-02  2.170e-01
   5.490e-01 -2.090e-01  1.210e-01 -4.150e-01  2.730e-01  4.600e-02
   5.720e-01  1.059e+00  2.520e-01  6.900e-01]
 [ 6.000e-03  3.470e-01  8.150e-01  9.000e-03  6.700e-01 -4.040e-01
  -5.010e-01 -5.870e-01  7.490e-01  9.870e-01 -4.400e-02  1.990e-01
   5.430e-01 -2.210e-01  1.090e-01 -4.310e-01  2.100e-01 -5.000e-03
   5.260e-01  1.006e+00  1.910e-01  6.690e-01]]
[[-0.801 -0.286  0.204  0.942 -1.547  0.013 -0.101  0.249 -0.491 -0.139
   0.741 -1.829 -0.924 -0.     0.64   0.158 -0.512  0.718  0.585 -0.154
   1.549]
 [-0.7   -0.302  0.222  0.986 -1.578 -0.004 -0.049  0.228 -0.505 -0.144
   0.769 -1.848 -0.936  0.013  0.644  0.38  -0.321  0.937  0.818  0.053
   1.753]]
320
[0.977 0.993 0.866]
[-780855.7 -615704.2 -165151.5 -777660.5 -615380.2 -162280.3]
[[-1.000e-03 -8.

410
[1.    0.993 1.   ]
[-779681.  -615504.1 -164176.9 -777660.5 -615380.2 -162280.3]
[[-1.000e-03 -3.400e-02  8.420e-01  4.100e-02  7.140e-01 -3.490e-01
  -4.560e-01 -5.470e-01  7.600e-01  9.900e-01 -3.400e-02  2.170e-01
   5.490e-01 -2.070e-01  1.300e-01 -4.150e-01  2.750e-01  6.000e-02
   5.800e-01  1.063e+00  2.530e-01  6.900e-01]
 [ 6.000e-03  3.470e-01  8.150e-01  9.000e-03  6.700e-01 -4.040e-01
  -5.010e-01 -5.870e-01  7.490e-01  9.870e-01 -4.400e-02  1.990e-01
   5.430e-01 -2.210e-01  1.090e-01 -4.310e-01  2.100e-01 -5.000e-03
   5.260e-01  1.006e+00  1.910e-01  6.690e-01]]
[[-0.811 -0.293  0.202  0.963 -1.532  0.006 -0.077  0.23  -0.49  -0.138
   0.739 -1.795 -0.917 -0.006  0.661  0.202 -0.447  0.788  0.65  -0.11
   1.541]
 [-0.7   -0.302  0.222  0.986 -1.578 -0.004 -0.049  0.228 -0.505 -0.144
   0.769 -1.848 -0.936  0.013  0.644  0.38  -0.321  0.937  0.818  0.053
   1.753]]
420
[1.    0.993 1.   ]
[-779569.2 -615454.2 -164115.  -777660.5 -615380.2 -162280.3]
[[-1.000e-03 -2.8

...

1520
[0.996 0.993 1.   ]
[-777952.8 -615331.1 -162621.7 -777660.5 -615380.2 -162280.3]
[[-0.007  0.245  0.822  0.032  0.681 -0.399 -0.485 -0.582  0.763  0.993
  -0.044  0.204  0.54  -0.212  0.117 -0.426  0.257  0.028  0.571  1.038
   0.229  0.686]
 [ 0.006  0.347  0.815  0.009  0.67  -0.404 -0.501 -0.587  0.749  0.987
  -0.044  0.199  0.543 -0.221  0.109 -0.431  0.21  -0.005  0.526  1.006
   0.191  0.669]]
[[-0.756 -0.29   0.216  0.979 -1.528  0.026 -0.076  0.246 -0.489 -0.134
   0.743 -1.816 -0.885  0.014  0.639  0.343 -0.328  0.893  0.768  0.019
   1.55 ]
 [-0.7   -0.302  0.222  0.986 -1.578 -0.004 -0.049  0.228 -0.505 -0.144
   0.769 -1.848 -0.936  0.013  0.644  0.38  -0.321  0.937  0.818  0.053
   1.753]]
1530
[0.912 0.993 1.   ]
[-777894.2 -615264.2 -162630.  -777660.5 -615380.2 -162280.3]
[[-0.007  0.245  0.823  0.033  0.685 -0.397 -0.488 -0.582  0.762  0.993
  -0.042  0.202  0.537 -0.211  0.114 -0.427  0.256  0.027  0.568  1.042
   0.227  0.697]
 [ 0.006  0.347  0.815  0.009  0.

1640
[1.    0.993 1.   ]
[-777867.  -615233.  -162634.1 -777660.5 -615380.2 -162280.3]
[[-0.007  0.256  0.816  0.033  0.69  -0.389 -0.486 -0.581  0.751  0.986
  -0.041  0.203  0.54  -0.217  0.109 -0.429  0.243  0.021  0.567  1.046
   0.224  0.694]
 [ 0.006  0.347  0.815  0.009  0.67  -0.404 -0.501 -0.587  0.749  0.987
  -0.044  0.199  0.543 -0.221  0.109 -0.431  0.21  -0.005  0.526  1.006
   0.191  0.669]]
[[-0.734 -0.292  0.208  0.931 -1.53   0.003 -0.042  0.241 -0.49  -0.147
   0.731 -1.799 -0.907  0.009  0.657  0.356 -0.329  0.907  0.802  0.028
   1.546]
 [-0.7   -0.302  0.222  0.986 -1.578 -0.004 -0.049  0.228 -0.505 -0.144
   0.769 -1.848 -0.936  0.013  0.644  0.38  -0.321  0.937  0.818  0.053
   1.753]]
1650
[0.991 0.993 0.996]
[-777900.7 -615301.5 -162599.2 -777660.5 -615380.2 -162280.3]
[[-0.007  0.261  0.816  0.033  0.688 -0.391 -0.483 -0.58   0.748  0.984
  -0.039  0.2    0.539 -0.219  0.108 -0.431  0.245  0.02   0.563  1.048
   0.22   0.69 ]
 [ 0.006  0.347  0.815  0.009  0.

1760
[1.    0.993 1.   ]
[-777761.8 -615148.  -162613.8 -777660.5 -615380.2 -162280.3]
[[-0.008  0.265  0.808  0.03   0.682 -0.396 -0.49  -0.591  0.754  0.983
  -0.04   0.2    0.539 -0.211  0.114 -0.43   0.247  0.022  0.55   1.032
   0.223  0.696]
 [ 0.006  0.347  0.815  0.009  0.67  -0.404 -0.501 -0.587  0.749  0.987
  -0.044  0.199  0.543 -0.221  0.109 -0.431  0.21  -0.005  0.526  1.006
   0.191  0.669]]
[[-0.737 -0.293  0.177  0.973 -1.55   0.028 -0.072  0.234 -0.485 -0.147
   0.736 -1.811 -0.908  0.016  0.63   0.373 -0.338  0.903  0.808  0.045
   1.565]
 [-0.7   -0.302  0.222  0.986 -1.578 -0.004 -0.049  0.228 -0.505 -0.144
   0.769 -1.848 -0.936  0.013  0.644  0.38  -0.321  0.937  0.818  0.053
   1.753]]
1770
[1.    0.993 1.   ]
[-777828.6 -615230.7 -162597.9 -777660.5 -615380.2 -162280.3]
[[-0.009  0.267  0.81   0.026  0.68  -0.393 -0.492 -0.586  0.756  0.982
  -0.038  0.199  0.536 -0.214  0.113 -0.43   0.249  0.017  0.549  1.032
   0.221  0.694]
 [ 0.006  0.347  0.815  0.009  0.

1870
[1.    0.993 1.   ]
[-777988.3 -615363.4 -162624.8 -777660.5 -615380.2 -162280.3]
[[-0.008  0.288  0.804  0.023  0.677 -0.391 -0.498 -0.58   0.747  0.986
  -0.042  0.207  0.541 -0.218  0.111 -0.432  0.238  0.015  0.547  1.029
   0.213  0.685]
 [ 0.006  0.347  0.815  0.009  0.67  -0.404 -0.501 -0.587  0.749  0.987
  -0.044  0.199  0.543 -0.221  0.109 -0.431  0.21  -0.005  0.526  1.006
   0.191  0.669]]
[[-0.729 -0.288  0.188  0.977 -1.526  0.007 -0.074  0.23  -0.474 -0.146
   0.751 -1.812 -0.924 -0.002  0.645  0.338 -0.333  0.903  0.79   0.031
   1.551]
 [-0.7   -0.302  0.222  0.986 -1.578 -0.004 -0.049  0.228 -0.505 -0.144
   0.769 -1.848 -0.936  0.013  0.644  0.38  -0.321  0.937  0.818  0.053
   1.753]]
1880
[0.996 0.993 0.996]
[-778047.  -615416.6 -162630.4 -777660.5 -615380.2 -162280.3]
[[-0.008  0.287  0.806  0.019  0.674 -0.395 -0.497 -0.58   0.747  0.99
  -0.039  0.203  0.539 -0.218  0.112 -0.433  0.239  0.016  0.547  1.034
   0.214  0.691]
 [ 0.006  0.347  0.815  0.009  0.6

1990
[0.986 0.993 1.   ]
[-777920.9 -615340.7 -162580.2 -777660.5 -615380.2 -162280.3]
[[-0.007  0.3    0.797  0.013  0.672 -0.401 -0.498 -0.578  0.754  0.98
  -0.039  0.208  0.544 -0.21   0.11  -0.424  0.241  0.015  0.547  1.033
   0.215  0.698]
 [ 0.006  0.347  0.815  0.009  0.67  -0.404 -0.501 -0.587  0.749  0.987
  -0.044  0.199  0.543 -0.221  0.109 -0.431  0.21  -0.005  0.526  1.006
   0.191  0.669]]
[[-0.762 -0.299  0.226  0.966 -1.538  0.011 -0.043  0.237 -0.483 -0.141
   0.748 -1.821 -0.903  0.009  0.641  0.377 -0.309  0.915  0.785  0.05
   1.561]
 [-0.7   -0.302  0.222  0.986 -1.578 -0.004 -0.049  0.228 -0.505 -0.144
   0.769 -1.848 -0.936  0.013  0.644  0.38  -0.321  0.937  0.818  0.053
   1.753]]
